<a href="https://colab.research.google.com/github/ulung3ko/deep-learning-assignment-hyperparameter-tuning/blob/main/hyperparameter_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install dan Import beberapa library yang dibutuhkan pada tugas ini

In [1]:
!pip install yfinance
!pip install yahoofinancials
!pip install scikeras

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.9/51.9 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for yahoofinancials: filename=yahoofinancials-1.20-py3-none-any.whl size=38618 sha256=95cdd1ce73e2cb21bb529332f4827ae805cf98c84c5747e1b43cab7a77d46596
  Stored in directory: /root/.cache/pip/wheels/c6/bc/18/da12b3c280d31beeac0c0f491c8f370da7c1473a56d7d4b5f3
Successfully built yahoofinancials


In [2]:
# Import Library
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
import yfinance as yf
from yahoofinancials import YahooFinancials
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import GridSearchCV

# Unduh dataset yang sesuai daengan UTS sebelumnya, yaitu data saham Samsung

In [3]:
samsung_df = yf.download('005930.KS',
                      start='2018-12-31',
                      end='2025-05-29',
                      progress=False,
)
print(samsung_df.head())

/tmp/ipython-input-3-1569164090.py:1: FutureWarning: YF.download() has changed argument auto_adjust default to True
  samsung_df = yf.download('005930.KS',


Price              Close          High           Low          Open    Volume
Ticker         005930.KS     005930.KS     005930.KS     005930.KS 005930.KS
Date                                                                        
2019-01-02  32866.582031  33417.892440  32696.948059  33417.892440   7847664
2019-01-03  31891.183594  32696.944881  31763.958127  32484.902437  12471493
2019-01-04  31763.960938  31891.186415  31255.059027  31763.960937  14108958
2019-01-07  32866.582031  32993.807510  32060.820665  32230.454637  12748997
2019-01-08  32315.275391  33248.262344  32188.049897  32230.458395  12756554


# Perbaiki nama kolom pada dataset yang baru saja di download

In [4]:
samsung_df.info()
samsung_df.columns = samsung_df.columns.get_level_values(0)
print(samsung_df.columns.tolist())
print(samsung_df.head())
print(f"Tipe Index Tanggal: {type(samsung_df.index)}, Nama Index: {samsung_df.index.name}, Tipe Data Index: {samsung_df.index.dtype}")

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1572 entries, 2019-01-02 to 2025-05-28
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   (Close, 005930.KS)   1572 non-null   float64
 1   (High, 005930.KS)    1572 non-null   float64
 2   (Low, 005930.KS)     1572 non-null   float64
 3   (Open, 005930.KS)    1572 non-null   float64
 4   (Volume, 005930.KS)  1572 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 73.7 KB
['Close', 'High', 'Low', 'Open', 'Volume']
Price              Close          High           Low          Open    Volume
Date                                                                        
2019-01-02  32866.582031  33417.892440  32696.948059  33417.892440   7847664
2019-01-03  31891.183594  32696.944881  31763.958127  32484.902437  12471493
2019-01-04  31763.960938  31891.186415  31255.059027  31763.960937  14108958
2019-01-07  32866.582031  32993.807510  32060.8

# Pilih variabel X dan y untuk modelling (fitur dan target)

In [5]:
feature_columns = ['Open', 'High', 'Low', 'Close', 'Volume']
target_column = 'Close'

data_to_process = samsung_df[feature_columns].copy()
print(f"Fitur yang digunakan: {feature_columns}")
print(f"Target prediksi: {target_column}")
print("Shape data_to_process:", data_to_process.shape)

Fitur yang digunakan: ['Open', 'High', 'Low', 'Close', 'Volume']
Target prediksi: Close
Shape data_to_process: (1572, 5)
